In [ ]:
#THESE SCRIPTS TAKE US FROM RAW DATA TO THE TABLES WE HAVE IN OUR SCHEMA - SEE movie_db_model.png

In [1]:
import pandas as pd
import numpy as np
from itertools import chain

c:\python37\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
df_title_basics = pd.read_csv('raw_data/title_basics.tsv', sep='\t')
print(list(df_title_basics))
df_title_ratings = pd.read_csv('raw_data/title_ratings.tsv', sep='\t')
print(list(df_title_ratings))

c:\python37\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres']
['tconst', 'averageRating', 'numVotes']


In [3]:
df_title_basics=df_title_basics.merge(df_title_ratings,on='tconst')

In [4]:
#Rename title columns
df_title_basics.columns=['title_id', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres', 'averageRating', 'numVotes']

#Replace all comman with semicolons to enable database import later
df_title_basics['genres'] = df_title_basics['genres'].str.replace(',', ';')

#Filter out null startYear values
df_title_basics = df_title_basics.loc[~df_title_basics['startYear'].isin(['\\N'])]

#Change votes and rating to numeric data types
df_title_basics['numVotes'] = df_title_basics['numVotes'].astype(str).astype(int)
df_title_basics['averageRating'] = df_title_basics['averageRating'].astype(str).astype(float)

#Sort descending by rating
df_title_basics = df_title_basics.sort_values('averageRating', ascending=False)

#then by number of votes
df_title_basics = df_title_basics.sort_values('numVotes', ascending=False)

#Drop duplicates
df_title_basics = df_title_basics.drop_duplicates(subset=['title_id'])

#Cut out all but the top 10000 records
df_title_basics = df_title_basics.head(5000)

#Then output to csv
print(df_title_basics.head())
df_title_basics.to_csv('app_data/title.csv', index=False)

         title_id titleType              primaryTitle  \
79660   tt0111161     movie  The Shawshank Redemption   
236682  tt0468569     movie           The Dark Knight   
481204  tt1375666     movie                 Inception   
95608   tt0137523     movie                Fight Club   
79446   tt0110912     movie              Pulp Fiction   

                   originalTitle  isAdult startYear endYear runtimeMinutes  \
79660   The Shawshank Redemption        0      1994      \N            142   
236682           The Dark Knight        0      2008      \N            152   
481204                 Inception        0      2010      \N            148   
95608                 Fight Club        0      1999      \N            139   
79446               Pulp Fiction        0      1994      \N            154   

                         genres  averageRating  numVotes  
79660                     Drama            9.3   2016942  
236682       Action;Crime;Drama            9.0   1985076  
481204  Act

In [10]:
#print column headers for manual population in MySQL workbench
print(list(df_title_basics))

['title_id', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres']
